In [66]:
import pyomo.environ as pe
import pyomo.opt as po
import pandas as pd
import math


In [68]:
df_stores = pd.read_excel(
    "Python Data Full Scale.xlsx",
    sheet_name="Stores",
    usecols=["Store Number"]
)

store_list = (
    df_stores["Store Number"]
    .dropna()
    .astype(str)
    .tolist()
)

# ------------------------------------------------------------------
# 2.  Hard-coded outlet list (as given)
# ------------------------------------------------------------------
outlet_list = ["2540", "2564", "2565", "2583", "2644", "2645"]

# ------------------------------------------------------------------
# 3.  Read SKU List ▸ SKU → sku_list
# ------------------------------------------------------------------
df_sku = pd.read_excel(
    "Python Data Full Scale.xlsx",
    sheet_name="SKU List",
    usecols=["SKU"]
)

sku_list = (
    df_sku["SKU"]
    .dropna()
    .astype(str)     # keep as str in case SKUs have leading zeros
    .tolist()
)

# Optional sanity check
print(f"{len(store_list)} stores →", store_list[:10], "…")
print(f"{len(outlet_list)} outlets →", outlet_list)
print(f"{len(sku_list)} SKUs   →", sku_list[:10], "…")

171 stores → ['2623', '2621', '2651', '2671', '2610', '2684', '2635', '2620', '2570', '2646'] …
6 outlets → ['2540', '2564', '2565', '2583', '2644', '2645']
848 SKUs   → ['1117915', '1118752', '1115853', '1115852', '1117930', '1115656', '1117926', '1117923', '1115733', '1115792'] …


In [70]:

# --- Model, Sets, Parameters ---
model = pe.ConcreteModel()
model.S = pe.Set(initialize=store_list)    # Retail stores (i)
model.O = pe.Set(initialize=outlet_list)   # Outlet stores (j)
model.K = pe.Set(initialize=sku_list)      # SKUs (k)

In [72]:
 # Stock[i,k]

df = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="Stock")

stock_dict = {(str(row['Store']), str(row['SKU'])): int(row['Stock']) for idx, row in df.iterrows()}


model.Stock = pe.Param(
    model.S, model.K,
    initialize=stock_dict,
    within=pe.NonNegativeIntegers,
    default=0       # <-- any (i,k) not in stock_dict is treated as 0
)

print("Number of (store, SKU) pairs:", len(stock_dict))


Number of (store, SKU) pairs: 12997


In [73]:
# Cap[j]


df_cap = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="Outlet Capacity")


cap_dict = {str(row['Store']): int(row['February Capacity (Delisted)']) for idx, row in df_cap.iterrows()}

print("Number of outlets in cap_dict:", len(cap_dict))
print("cap_dict:", cap_dict)

model.Cap = pe.Param(model.O, initialize=cap_dict)


Number of outlets in cap_dict: 6
cap_dict: {'2540': 30376, '2564': 24704, '2565': 27918, '2583': 20768, '2644': 25768, '2645': 15526}


In [76]:
 # PSize[k]

df_psize = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="Pallet")
psize_dict = {str(row['articlecode']): int(row['PalletQuantiyPieces']) for idx, row in df_psize.iterrows()}

print("Number of SKUs in psize_dict:", len(psize_dict))


model.PSize = pe.Param(model.K, initialize=psize_dict)



Number of SKUs in psize_dict: 848


In [78]:
# r[k]

df_rev = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="Revenue")
revenue_dict = {
    str(int(row['articlecode'])): float(row['revenue'])      # <-- int() kills the “.0”
    for _, row in df_rev.iterrows()
}

print("Number of SKUs in revenue_dict:", len(revenue_dict))


model.r = pe.Param(model.K, initialize=revenue_dict) 



Number of SKUs in revenue_dict: 848


In [79]:
          
model.c_p = pe.Param(initialize=175)                                # €/pallet
model.f = pe.Param(initialize=60)                            # €/truck 
       
model.M = pe.Param(initialize=33)                           # Big M

In [82]:
# --- Decision variables ---
model.x = pe.Var(model.S, model.O, model.K, domain=pe.NonNegativeIntegers)
model.p = pe.Var(model.S, model.O, domain=pe.NonNegativeReals)     
model.w = pe.Var(model.S, model.O, domain=pe.Binary)
model.s = pe.Var(model.S, model.K, domain=pe.Binary)


In [84]:
# Objective: maximize net profit = revenue – transport cost per pallet – fixed truck cost

revenue_expr = sum(model.r[k] * model.x[i, j, k] 
                   for i in model.S for j in model.O for k in model.K)

cost_expr    = sum(model.c_p * model.p[i, j] + model.f * model.w[i, j] 
                   for i in model.S for j in model.O)


model.obj    = pe.Objective(expr=revenue_expr - cost_expr, sense= pe.maximize)

In [85]:
# ------------------------------------------------------------
# (2) Supply constraint: ship all OR none of each SKU from each store
#    ∑_{j∈O} x[i,j,k] = Stock[i,k] · s[i,k]     ∀ i∈S, k∈K
# ------------------------------------------------------------
model.SupplyConstraint = pe.ConstraintList()
for i in model.S:
    for k in model.K:
        expr = sum(model.x[i, j, k] for j in model.O) == model.Stock[i, k] * model.s[i, k]
        model.SupplyConstraint.add(expr)

In [86]:
## ------------------------------------------------------------
# (3) Outlet capacity: Σ_{k∈K} Σ_{i∈S} x[i,j,k] ≤ Cap[j]   ∀ j∈O
# ------------------------------------------------------------
model.OutletCapacityConstraint = pe.ConstraintList()
for j in model.O:                                  # every outlet j
    expr = sum(model.x[i, j, k]                    # flow i→j of SKU k
               for i in model.S                    #   sum over all retail stores
               for k in model.K)                   #   … and all SKUs
    model.OutletCapacityConstraint.add(expr <= model.Cap[j])


In [87]:
# ------------------------------------------------------------
# (4) Truck capacity on route (i→j):
#    p[i,j] ≤ M · w[i,j]                         ∀ i∈S, j∈O
# ------------------------------------------------------------
model.TruckCapacityConstraint = pe.ConstraintList()
for i in model.S:
    for j in model.O:
        expr = model.p[i, j] <= model.M * model.w[i, j]
        model.TruckCapacityConstraint.add(expr)

In [88]:
# ------------------------------------------------------------
# (5) Pallet-mixing (link pallets to units shipped):
#    ∑_{k∈K} x[i,j,k] / PSize[k] = p[i,j]        ∀ i∈S, j∈O
# ------------------------------------------------------------
model.PalletMixingConstraint = pe.ConstraintList()
for i in model.S:
    for j in model.O:
        expr = sum(model.x[i, j, k] / model.PSize[k] for k in model.K) == model.p[i, j]
        model.PalletMixingConstraint.add(expr)


In [94]:
solver = po.SolverFactory('gurobi')
solver.options['TimeLimit'] = 180  # Set time limit to 180 seconds
result = solver.solve(model, tee=True)

print(result.solver.status)
print(result.solver.termination_condition)
print("Objective value = " + str(pe.value(model.obj)))

# After solving, calculate rounded cost
if result.solver.termination_condition == pe.TerminationCondition.optimal:
    rounded_cost = sum(model.c_p * math.ceil(pe.value(model.p[i, j])) 
                       for i in model.S for j in model.O)
    print(f"Cost with rounded pallets: {rounded_cost}")

Set parameter Username
Set parameter LicenseID to value 2617608
Academic license - for non-commercial use only - expires 2026-02-03
Read LP format model from file C:\Users\User\AppData\Local\Temp\tmpqqyged18.pyomo.lp
Reading time = 3.19 seconds
x1: 147066 rows, 885097 columns, 2626219 nonzeros
Set parameter TimeLimit to value 180
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  180

Optimize a model with 147066 rows, 885097 columns and 2626219 nonzeros
Model fingerprint: 0x731acd57
Variable types: 1026 continuous, 884071 integer (14023 binary)
Coefficient statistics:
  Matrix range     [4e-05, 9e+02]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+04, 3e+04]
Found heuristic solution: objective -0.0000000
Presolve remov

In [96]:

if result.solver.termination_condition == pe.TerminationCondition.optimal:
    # Calculate revenue (remove .value from model.r[k] since it's already a number)
    revenue_value = sum(model.r[k] * pe.value(model.x[i, j, k]) 
                       for i in model.S for j in model.O for k in model.K)
    
    # Calculate cost with rounded pallets (remove .value from model.c_p and model.f if they're numbers)
    rounded_cost = sum(model.c_p * math.ceil(pe.value(model.p[i, j])) + 
                      model.f * pe.value(model.w[i, j]) 
                      for i in model.S for j in model.O)
    
    # Calculate objective with rounded pallets
    rounded_obj_value = revenue_value - rounded_cost
    
    print(f"Objective value with rounded pallets: {rounded_obj_value:.2f}")
    
else:
    print("Model did not solve to optimality")

Objective value with rounded pallets: 1807872.92


In [98]:
# Print all x[i, j, k] values (units shipped from store i to outlet j, SKU k)
for i in model.S:
    for j in model.O:
        for k in model.K:
            val = pe.value(model.x[i, j, k])
            if val > 1e-6:   # only print nonzero shipments 
                print(f"x[{i},{j},{k}] = {val}")


x[2623,2540,1117915] = 154.0
x[2623,2540,1118752] = 6.0
x[2623,2540,1115853] = 4.0
x[2623,2540,1115852] = 14.0
x[2623,2540,1117930] = 12.0
x[2623,2540,1115656] = 8.0
x[2623,2540,1117926] = 61.0
x[2623,2540,1117923] = 27.0
x[2623,2540,1115733] = 10.0
x[2623,2540,1115792] = 9.0
x[2623,2540,1115794] = 4.0
x[2623,2540,1117932] = 29.0
x[2623,2540,1117929] = 17.0
x[2623,2540,1116387] = 12.0
x[2623,2540,1118065] = 11.0
x[2623,2540,1118636] = 4.0
x[2623,2540,1118070] = 1.0
x[2623,2540,1116881] = 6.0
x[2623,2540,1116421] = 32.0
x[2623,2540,1118074] = 1.0
x[2623,2540,1115854] = 3.0
x[2623,2540,1115056] = 13.0
x[2623,2540,1116313] = 17.0
x[2623,2540,1115718] = 24.0
x[2623,2540,1116218] = 11.0
x[2623,2540,1114639] = 2.0
x[2623,2540,1115021] = 15.0
x[2623,2540,1115732] = 4.0
x[2623,2540,1108241] = 2.0
x[2623,2540,1108245] = 3.0
x[2623,2540,1116422] = 6.0
x[2623,2540,1109908] = 15.0
x[2623,2540,1114638] = 6.0
x[2623,2540,1115683] = 6.0
x[2623,2540,1116221] = 2.0
x[2623,2540,1113266] = 7.0
x[2623,254

In [102]:
for (i, k) in stock_dict.keys():
    shipped = sum(pe.value(model.x[i, j, k]) for j in model.O)
    stock = stock_dict[(i, k)]
    print(f"Store {i}, SKU {k}: shipped = {shipped}, stock = {stock}")
    if shipped > stock:
        print("WARNING: More shipped than in stock!")

Store 2502, SKU 1117915: shipped = 115.0, stock = 115
Store 2502, SKU 1116042: shipped = 54.0, stock = 54
Store 2502, SKU 1117930: shipped = 33.0, stock = 33
Store 2502, SKU 1116434: shipped = 27.0, stock = 27
Store 2502, SKU 1115022: shipped = 26.0, stock = 26
Store 2502, SKU 1117923: shipped = 22.0, stock = 22
Store 2502, SKU 1116367: shipped = 21.0, stock = 21
Store 2502, SKU 1115021: shipped = 18.0, stock = 18
Store 2502, SKU 1101310: shipped = 17.0, stock = 17
Store 2502, SKU 1116313: shipped = 17.0, stock = 17
Store 2502, SKU 1116322: shipped = 16.0, stock = 16
Store 2502, SKU 1117926: shipped = 16.0, stock = 16
Store 2502, SKU 1118070: shipped = 16.0, stock = 16
Store 2502, SKU 1116421: shipped = 13.0, stock = 13
Store 2502, SKU 1116726: shipped = 13.0, stock = 13
Store 2502, SKU 1117931: shipped = 12.0, stock = 12
Store 2502, SKU 1116387: shipped = 11.0, stock = 11
Store 2502, SKU 1115853: shipped = 10.0, stock = 10
Store 2502, SKU 1118065: shipped = 10.0, stock = 10
Store 2502

In [104]:
# ----------------------------------------------------------
# Build summary table: Store | SKU | Shipped | Stock | Over-shipped
# ----------------------------------------------------------
rows = []
for (i, k), stock_val in stock_dict.items():            # only pairs that have stock data
    shipped_val = sum(pe.value(model.x[i, j, k]) for j in model.O)
    rows.append({
        "Store":        i,
        "SKU":          k,
        "Shipped":      shipped_val,
        "Stock":        stock_val,
        "Over-shipped": shipped_val > stock_val
    })

df_summary = pd.DataFrame(rows)

# ----------------------------------------------------------
# Export to Excel   (second report)
# ----------------------------------------------------------
file_path_summary = "shipped_vs_stock.xlsx"             
df_summary.to_excel(file_path_summary, index=False)

In [112]:

# Print number of pallets shipped per route
for i in model.S:
    for j in model.O:
        pallets = pe.value(model.p[i, j])
        if pallets > 1e-6:
            print(f"p[{i},{j}] = {pallets}")


rows = []
for i in model.S:
    for j in model.O:
        pallets = pe.value(model.p[i, j]) or 0
        if pallets > 1e-6:                    # keep only routes that ship pallets
            rows.append({"Store": i,
                         "Outlet": j,
                         "Pallets": pallets})



p[2623,2540] = 1.908317709626463
p[2621,2645] = 1.5310242168409995
p[2651,2540] = 2.8862499979589313
p[2671,2644] = 0.8166357484578047
p[2610,2583] = 0.5661961959080282
p[2635,2564] = 3.3300881484376483
p[2620,2644] = 2.211268449546474
p[2570,2564] = 0.6187701851938444
p[2646,2565] = 0.8213430861448926
p[2602,2540] = 1.580727033114729
p[2530,2583] = 1.6590128638150317
p[2656,2583] = 1.9808917488555344
p[2520,2645] = 0.5888657095619105
p[2611,2645] = 1.317808779948778
p[2679,2565] = 0.37285810982991524
p[2587,2565] = 0.9520429138825243
p[2546,2564] = 0.5482228523376088
p[2591,2644] = 1.5315591147119318
p[2665,2583] = 1.960380532298421
p[2663,2564] = 0.6686823431404544
p[2577,2564] = 0.9586261177736727
p[2598,2645] = 1.8229317487386605
p[2626,2583] = 1.0402439582268477
p[2622,2540] = 1.6991444570365428
p[2624,2645] = 0.8989977766480325
p[2560,2645] = 0.63336216967222
p[2614,2645] = 1.035560118445081
p[2657,2540] = 0.062366779007098166
p[2657,2583] = 0.9113254920887499
p[2511,2644] = 1.13

In [110]:

# Print number of pallets shipped per route (rounded up)
for i in model.S:
    for j in model.O:
        pallets = pe.value(model.p[i, j])
        if pallets > 1e-6:
            rounded_pallets = math.ceil(pallets)
            print(f"p[{i},{j}] = {rounded_pallets}")

# Create DataFrame with rounded up pallet values
rows = []
for i in model.S:
    for j in model.O:
        pallets = pe.value(model.p[i, j]) or 0
        if pallets > 1e-6:                    # keep only routes that ship pallets
            rounded_pallets = math.ceil(pallets)
            rows.append({"Store": i,
                         "Outlet": j,
                         "Pallets": rounded_pallets})

df_pallets = pd.DataFrame(rows)

# -------------------------------------------------
# Export to Excel
# -------------------------------------------------
file_path_pallets = "pallets_by_route.xlsx"
df_pallets.to_excel(file_path_pallets, index=False)

p[2623,2540] = 2
p[2621,2645] = 2
p[2651,2540] = 3
p[2671,2644] = 1
p[2610,2583] = 1
p[2635,2564] = 4
p[2620,2644] = 3
p[2570,2564] = 1
p[2646,2565] = 1
p[2602,2540] = 2
p[2530,2583] = 2
p[2656,2583] = 2
p[2520,2645] = 1
p[2611,2645] = 2
p[2679,2565] = 1
p[2587,2565] = 1
p[2546,2564] = 1
p[2591,2644] = 2
p[2665,2583] = 2
p[2663,2564] = 1
p[2577,2564] = 1
p[2598,2645] = 2
p[2626,2583] = 2
p[2622,2540] = 2
p[2624,2645] = 1
p[2560,2645] = 1
p[2614,2645] = 2
p[2657,2540] = 1
p[2657,2583] = 1
p[2511,2644] = 2
p[2666,2645] = 1
p[2627,2564] = 2
p[2615,2645] = 2
p[2638,2540] = 2
p[2628,2564] = 2
p[2569,2644] = 2
p[2593,2645] = 2
p[2664,2564] = 1
p[2664,2583] = 1
p[2662,2564] = 2
p[2518,2564] = 1
p[2539,2583] = 1
p[2648,2540] = 1
p[2532,2583] = 1
p[2634,2645] = 2
p[2588,2540] = 2
p[2592,2644] = 2
p[2582,2565] = 1
p[2633,2564] = 2
p[2625,2540] = 2
p[2650,2583] = 2
p[2600,2540] = 3
p[2675,2565] = 1
p[2641,2583] = 2
p[2537,2564] = 2
p[2578,2583] = 1
p[2661,2644] = 2
p[2596,2540] = 1
p[2533,2540] =

In [351]:
# -------------------------------------------------
# Build table: Store | Outlet | SKU | Shipped
# -------------------------------------------------
rows = []
for i in model.S:
    for j in model.O:
        for k in model.K:
            qty = pe.value(model.x[i, j, k]) or 0
            if qty > 1e-6:        # keep only positive flows
                rows.append({"Store": i, "Outlet": j, "SKU": k, "Shipped": qty})

df_ship = pd.DataFrame(rows)

# -------------------------------------------------
# Export to Excel
# -------------------------------------------------
file_path = "shipments_by_outlet.xlsx"
df_ship.to_excel(file_path, index=False)



In [352]:


# Print route activation variable
for i in model.S:
    for j in model.O:
        wval = pe.value(model.w[i, j])
        if wval > 0.5:
            print(f"w[{i},{j}] = {int(wval)}")

w[2623,2540] = 1
w[2621,2645] = 1
w[2651,2540] = 1
w[2671,2644] = 1
w[2610,2583] = 1
w[2635,2564] = 1
w[2620,2644] = 1
w[2570,2564] = 1
w[2646,2565] = 1
w[2602,2540] = 1
w[2530,2583] = 1
w[2656,2583] = 1
w[2520,2645] = 1
w[2611,2645] = 1
w[2679,2565] = 1
w[2587,2565] = 1
w[2546,2564] = 1
w[2591,2644] = 1
w[2665,2583] = 1
w[2663,2564] = 1
w[2577,2564] = 1
w[2598,2645] = 1
w[2626,2583] = 1
w[2622,2540] = 1
w[2624,2645] = 1
w[2560,2645] = 1
w[2614,2645] = 1
w[2657,2540] = 1
w[2657,2583] = 1
w[2511,2644] = 1
w[2666,2645] = 1
w[2627,2564] = 1
w[2615,2645] = 1
w[2638,2540] = 1
w[2628,2564] = 1
w[2569,2644] = 1
w[2593,2645] = 1
w[2664,2564] = 1
w[2664,2583] = 1
w[2662,2564] = 1
w[2518,2564] = 1
w[2539,2583] = 1
w[2648,2540] = 1
w[2532,2583] = 1
w[2634,2645] = 1
w[2588,2540] = 1
w[2592,2644] = 1
w[2582,2565] = 1
w[2633,2564] = 1
w[2625,2540] = 1
w[2650,2583] = 1
w[2600,2540] = 1
w[2675,2565] = 1
w[2641,2583] = 1
w[2537,2564] = 1
w[2578,2583] = 1
w[2661,2644] = 1
w[2596,2540] = 1
w[2533,2540] =

In [355]:


if result.solver.termination_condition in ("optimal", "feasible"):
    for (i, k) in stock_dict.keys():          # only pairs that actually have stock
        sval = model.s[i, k].value            # .value is None if never assigned
        if sval is not None and sval > 0.5:
            print(f"s[{i},{k}] = 1")         
else:
    print("No feasible solution – variables are uninitialized.")

s[2502,1117915] = 1
s[2502,1116042] = 1
s[2502,1117930] = 1
s[2502,1116434] = 1
s[2502,1115022] = 1
s[2502,1117923] = 1
s[2502,1116367] = 1
s[2502,1115021] = 1
s[2502,1101310] = 1
s[2502,1116313] = 1
s[2502,1116322] = 1
s[2502,1117926] = 1
s[2502,1118070] = 1
s[2502,1116421] = 1
s[2502,1116726] = 1
s[2502,1117931] = 1
s[2502,1116387] = 1
s[2502,1115853] = 1
s[2502,1118065] = 1
s[2502,1111807] = 1
s[2502,1114624] = 1
s[2502,1114854] = 1
s[2502,1116218] = 1
s[2502,1116348] = 1
s[2502,1116616] = 1
s[2502,1101307] = 1
s[2502,1116216] = 1
s[2502,1101306] = 1
s[2502,1114635] = 1
s[2502,1114638] = 1
s[2502,1115133] = 1
s[2502,1116422] = 1
s[2502,1117929] = 1
s[2502,1110004] = 1
s[2502,1112040] = 1
s[2502,1112043] = 1
s[2502,1112108] = 1
s[2502,1115056] = 1
s[2502,1115683] = 1
s[2502,1115718] = 1
s[2502,1116222] = 1
s[2502,1118752] = 1
s[2502,1109908] = 1
s[2502,1112027] = 1
s[2502,1112265] = 1
s[2502,1114639] = 1
s[2502,1114983] = 1
s[2502,1115656] = 1
s[2502,1116217] = 1
s[2502,1106456] = 1


In [357]:
for i in model.S:
    for k in model.K:
        shipped = sum(model.x[i, j, k].value for j in model.O if model.x[i, j, k].value is not None)
        if shipped > 1e-6:
            print(f"Store {i}, SKU {k}, shipped: {shipped}")

Store 2623, SKU 1117915, shipped: 154.0
Store 2623, SKU 1118752, shipped: 6.0
Store 2623, SKU 1115853, shipped: 4.0
Store 2623, SKU 1115852, shipped: 14.0
Store 2623, SKU 1117930, shipped: 12.0
Store 2623, SKU 1115656, shipped: 8.0
Store 2623, SKU 1117926, shipped: 61.0
Store 2623, SKU 1117923, shipped: 27.0
Store 2623, SKU 1115733, shipped: 10.0
Store 2623, SKU 1115792, shipped: 9.0
Store 2623, SKU 1115794, shipped: 4.0
Store 2623, SKU 1117932, shipped: 29.0
Store 2623, SKU 1117929, shipped: 17.0
Store 2623, SKU 1116387, shipped: 12.0
Store 2623, SKU 1118065, shipped: 11.0
Store 2623, SKU 1118636, shipped: 4.0
Store 2623, SKU 1118070, shipped: 1.0
Store 2623, SKU 1116881, shipped: 6.0
Store 2623, SKU 1116421, shipped: 32.0
Store 2623, SKU 1118074, shipped: 1.0
Store 2623, SKU 1115854, shipped: 3.0
Store 2623, SKU 1115056, shipped: 13.0
Store 2623, SKU 1116313, shipped: 17.0
Store 2623, SKU 1115718, shipped: 24.0
Store 2623, SKU 1116218, shipped: 11.0
Store 2623, SKU 1114639, shipped: 

In [158]:
supply_pairs = set(stock_dict.keys())
print("Supply constraint indices:", supply_pairs)

all_model_x_indices = set()
for i in model.S:
    for j in model.O:
        for k in model.K:
            all_model_x_indices.add((i, j, k))

#  if there are any (i, k) pairs in your variable domain that are NOT constrained:
for (i, j, k) in all_model_x_indices:
    if (i, k) not in supply_pairs:
        print(f"x[{i},{j},{k}] is NOT constrained by supply!")

Supply constraint indices: {('2584', '1117923'), ('2609', '1117926'), ('2628', '1116421'), ('2541', '12640'), ('2569', '1116423'), ('2622', '1115021'), ('2518', '1115733'), ('2588', '1114209'), ('2597', '1116313'), ('2512', '1116219'), ('2646', '1115678'), ('2552', '1105081'), ('2587', '1112699'), ('2646', '1112109'), ('2573', '1113445'), ('2509', '1113267'), ('2604', '1113266'), ('2597', '1115414'), ('2649', '1116727'), ('2555', '1115073'), ('2547', '1116182'), ('2599', '1104915'), ('2533', '1101311'), ('2638', '1116300'), ('2541', '1115435'), ('2503', '1118074'), ('2548', '1109518'), ('2603', '1114633'), ('2590', '1117926'), ('2570', '1117923'), ('2504', '1104772'), ('2515', '1112694'), ('2594', '1113463'), ('2637', '1117915'), ('2655', '1114856'), ('2657', '1115678'), ('2618', '1118065'), ('2515', '1112035'), ('2622', '1106563'), ('2671', '1117930'), ('2631', '1113327'), ('2571', '1118752'), ('2507', '1119411'), ('2539', '1105814'), ('2627', '1117915'), ('2627', '1116619'), ('2653',

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




x[2530,2564,1103618] is NOT constrained by supply!
x[2634,2644,1105755] is NOT constrained by supply!
x[2506,2583,1108982] is NOT constrained by supply!
x[2649,2565,1100676] is NOT constrained by supply!
x[2672,2564,1105689] is NOT constrained by supply!
x[2609,2645,1113523] is NOT constrained by supply!
x[2683,2644,1105142] is NOT constrained by supply!
x[2580,2645,1107628] is NOT constrained by supply!
x[2658,2644,1106565] is NOT constrained by supply!
x[2651,2540,1112091] is NOT constrained by supply!
x[2653,2565,1113955] is NOT constrained by supply!
x[2666,2540,1114421] is NOT constrained by supply!
x[2537,2564,1114437] is NOT constrained by supply!
x[2511,2583,1102715] is NOT constrained by supply!
x[2506,2645,1106422] is NOT constrained by supply!
x[2594,2564,1113702] is NOT constrained by supply!
x[2526,2645,1104649] is NOT constrained by supply!
x[2582,2644,1108825] is NOT constrained by supply!
x[2537,2645,1107346] is NOT constrained by supply!
x[2557,2644,14553] is NOT cons

In [365]:
import math

# Calculate total rounded up pallets
total_pallets_rounded = sum(math.ceil(pe.value(model.p[i, j])) for i in model.S for j in model.O)

# 1) variable cost with rounded up pallets (€/pallet)
var_cost_rounded = model.c_p * total_pallets_rounded

# 2) fixed cost (€/truck dispatched) - this stays the same
fixed_cost_rounded = model.f * sum(
    pe.value(model.w[i, j])                # 1 if truck used on route i→j
    for i in model.S for j in model.O
)

# 3) total cost with rounded pallets
total_cost_rounded = var_cost_rounded + fixed_cost_rounded

print(f"Total pallets shipped (rounded up)                    : {total_pallets_rounded:,}")
print(f"Variable transport cost with rounded pallets (€/pallet): {var_cost_rounded:,.2f}")
print(f"Fixed truck cost (€/truck)                             : {fixed_cost_rounded:,.2f}")
print(f"------------------------------------------------------------------")
print(f"Total transport cost with rounded pallets              : {total_cost_rounded:,.2f}")

Total pallets shipped (rounded up)                    : 262
Variable transport cost with rounded pallets (€/pallet): 45,850.00
Fixed truck cost (€/truck)                             : 9,660.00
------------------------------------------------------------------
Total transport cost with rounded pallets              : 55,510.00
